In [3]:
push!(LOAD_PATH, "/home/amir/Work/gdmrg")
import Tmp

In [38]:
reload("Tmp")
include("test.jl")

Test Passed

In [ ]:
Tmp.runstuff(8,100)

In [39]:
singlet = Vector{Complex128}([0, 1, -1, 0])
ketstate = kron(singlet, singlet)
mps = Tmp.MPS(4, 2, ketstate)
mpo = Tmp.MPO(4, 2)
Tmp.expectation_mpo(mps, mpo)

1×1×1 Array{Float64,3}:
[:, :, 1] =
 -6.0

In [31]:
mpo = Tmp.MPO(2,2)
for i=1:5
    println(mpo.matrices[1][:,1,:,i])
end

Complex{Float64}[0.0+0.0im 0.0+0.0im; 0.0+0.0im 0.0+0.0im]
Complex{Float64}[0.0+0.0im 0.0+0.0im; 1.0+0.0im 0.0+0.0im]
Complex{Float64}[0.0+0.0im 1.0+0.0im; 0.0+0.0im 0.0+0.0im]
Complex{Float64}[0.5+0.0im 0.0+0.0im; 0.0+0.0im -0.5+0.0im]
Complex{Float64}[1.0+0.0im 0.0+0.0im; 0.0+0.0im 1.0+0.0im]
